In [ ]:
from google import genai
from google.genai import types
import os, json, time, random, wave
from datetime import datetime
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# API KEY ROTATION
KEYS_FILE = "/content/drive/MyDrive/Alot1.txt" # change this with your txt file path containing the API keys line by line

with open(KEYS_FILE, "r") as f:
    API_KEYS = [k.strip() for k in f.readlines() if k.strip()]

assert API_KEYS, "❌ No API keys found!"

current_key_index = 0
client = None

def configure_key():
    global current_key_index, client
    client = genai.Client(api_key=API_KEYS[current_key_index])
    print(f"🔑 Using API key {current_key_index + 1}/{len(API_KEYS)}")

def rotate_key():
    global current_key_index
    current_key_index += 1
    if current_key_index >= len(API_KEYS):
        raise RuntimeError("🛑 All Gemini API keys exhausted!")
    configure_key()

configure_key()

# PATHS
DATASET_DIR = "/content/drive/MyDrive/teacher_dataset_large_updated"
VOICE_DIR = os.path.join(DATASET_DIR, "voices")
METADATA_FILE = os.path.join(DATASET_DIR, "metadata.jsonl")
os.makedirs(VOICE_DIR, exist_ok=True)

# mutlitple Voices males and females
VOICE_MAP = {
    "Puck": {"pitch": "medium-low"},
    "Leda": {"pitch": "medium"},
    "Zephyr": {"pitch": "low"},
    "Kore": {"pitch": "high"},
    "Charon": {"pitch": "deep"},
    "Aoede": {"pitch": "bright/melodic"},
    "Gacrux": {"pitch": "mature"},
    "Achird": {"pitch": "soft"},
    "Sulafat": {"pitch": "steady"},
    "Orus": {"pitch": "bright"}
}

VOICES = list(VOICE_MAP.keys())

# emotions control
STYLES = [
    "cheerful", "gentle", "energetic", "whispering", "slow",
    "authoritative", "playful", "calm", "excited", "sad",
    "surprised", "serious", "friendly", "cry", "enthusiastic",
    "soothing", "firm", "formal", "anxious", "curious",
    "mellow", "bright", "commanding", "kind", "animated", "laugh"
]


# Alot of topics
topics = [
    # --- SPACE & ASTRONOMY ---
    "Why do stars twinkle?", "How do rockets launch into space?", "Why is space silent?", "How do astronauts sleep in space?",
    "What is a shooting star really?", "Why doesn’t the Moon fall to Earth?", "How do planets stay in orbit?", "What is space made of?",
    "What is a space station?", "Why does Earth have seasons?", "How do satellites work?", "Why is Pluto not a planet?",
    "What is a supernova?", "How do we explore Mars?", "What is a space suit for?", "What is space dust?",
    "Why does the Sun shine?", "What happens inside the Sun?", "What is the edge of space?", "How do astronauts eat in space?",
    "How do black holes form?", "Why is the Milky Way spiral-shaped?", "What are Saturn's rings made of?", "How do telescopes see distant galaxies?",
    "Why is Mars red?", "What causes a lunar eclipse?", "How do we measure the distance to stars?", "What is a light-year?",
    "Why do comets have tails?", "How does the International Space Station get oxygen?", "What is dark matter?", "Why is Venus the hottest planet?",
    "How do we find exoplanets?", "What is a nebula?", "Why does the Moon have craters?", "How do astronauts use the bathroom in space?",
    "What is the Big Bang theory?", "How do solar flares affect Earth?", "Why is space black despite all the stars?", "How do stars die?",
    "What is a pulsar?", "How does gravity work in zero-G?", "Why do astronauts grow taller in space?", "What is the Oort Cloud?",
    "How do space probes send photos back to Earth?", "Why is Jupiter so big?", "What is a wormhole?", "How do we predict meteor showers?",
    "Why does the Moon look larger on the horizon?", "What is a solar wind?", "How do we shield spacecraft from radiation?",

    # --- NATURE & EARTH SCIENCE ---
    "Why does lightning strike?", "How do rainbows disappear?", "Why do rivers flow?", "How are caves formed?",
    "Why do waves crash?", "How do islands form?", "What causes geysers to erupt?", "Why do cliffs erode?",
    "How does fog form?", "Why do rivers curve?", "How does dew form?", "Why do rocks crack?",
    "What causes avalanches?", "Why do sinkholes appear?", "How does soil form?", "Why do dunes move?",
    "What causes mudslides?", "How do hot springs work?", "Why do rivers dry up?", "How does erosion shape land?",
    "How do volcanoes erupt?", "Why is the ocean salty?", "What causes earthquakes?", "How do glaciers move?",
    "Why is the sky blue?", "How does a hurricane form?", "What is the water cycle?", "Why are some rocks shiny?",
    "How do stalactites grow?", "Why do mountains have snow at the top?", "What causes a tsunami?", "How does lightning make thunder?",
    "Why do clouds look white?", "How do tides work?", "What is the ozone layer?", "Why does the wind blow?",
    "How do fossils form?", "What causes the Northern Lights?", "Why is the Earth's core hot?", "How do minerals grow?",
    "What is a tectonic plate?", "How do deserts stay dry?", "Why do some lakes turn pink?", "How does carbon dating work?",
    "What is an aquifer?", "Why is topsoil so important?", "How do canyons form?", "Why does magma turn into lava?",
    "What causes a double rainbow?", "How do quicksand pits form?", "Why do coral reefs need sunlight?",

    # --- ANIMAL KINGDOM ---
    "How do insects breathe?", "Why do birds sing?", "How do fish find food?", "Why do animals hibernate?",
    "How do snakes sense heat?", "Why do frogs croak?", "How do spiders climb walls?", "Why do animals shed fur?",
    "How do fish use fins?", "Why do animals have tails?", "How do butterflies drink?", "Why do owls fly silently?",
    "How do sharks sense movement?", "Why do animals have patterns?", "How do snails move?", "Why do animals play?",
    "How do insects see?", "Why do birds fluff feathers?", "How do animals recognize danger?", "How do ants communicate?",
    "Why do bees dance?", "How do bats fly in the dark?", "Why do cats have whiskers?", "How do chameleons change color?",
    "Why do penguins huddle?", "How do whales hold their breath?", "Why do fireflies glow?", "How do spiders spin webs?",
    "Why do elephants have big ears?", "How do salmon find their way home?", "Why do zebras have stripes?", "How do jellyfish sting?",
    "Why do kangaroos have pouches?", "How do birds migrate?", "Why do dogs sniff everything?", "How do octopuses camouflage?",
    "Why do giraffes have long necks?", "How do woodpeckers protect their brains?", "Why do camels have humps?", "How do crickets chirp?",
    "Why do wolves howl?", "How do squirrels remember where they hid nuts?", "Why do parrots mimic sounds?", "How do polar bears stay warm?",
    "Why do mosquitoes bite?", "How do bees make honey?", "Why do snakes shed skin?", "How do dolphins use sonar?",
    "Why do crabs walk sideways?", "How do turtles live so long?", "Why do horses sleep standing up?",

    # --- THE HUMAN BODY ---
    "How does skin heal?", "Why do we sneeze?", "Why do we get tired?", "How does our body fight germs?",
    "Why do we shiver?", "How does our body grow?", "Why do we laugh?", "How do muscles get stronger?",
    "Why do we feel pain?", "How does our body cool down?", "Why do we feel dizzy?", "How does our body digest food?",
    "Why do we cough?", "How do our eyes adjust to light?", "Why do we stretch?", "How does our immune system work?",
    "Why do we feel thirsty?", "Why does our heart beat faster when running?", "How does breathing slow down?",
    "Why do we get sleepy at night?", "How does the body store energy?", "Why do we blush?", "How does the body balance itself?",
    "Why do we hiccup after eating fast?", "How does the brain send messages?", "Why do we feel ticklish?", "How does the body repair bones?",
    "Why do we have fingerprints?", "How do we taste flavors?", "Why do we dream?", "How do we hear sound?",
    "Why do we have eyebrows?", "How do we remember things?", "Why do we get goosebumps?", "How does blood circulate?",
    "Why do we have different blood types?", "How do we smell scents?", "Why do we blink?", "How do lungs extract oxygen?",
    "Why do we have baby teeth?", "How do kidneys filter blood?", "Why do we get scars?", "How does the liver function?",
    "Why do we have fingernails?", "How do our vocal cords work?", "Why do we get brain freeze?", "How do we stay upright?",
    "Why do we yawn?", "How does hair grow?", "Why do we get sunburnt?", "How do we feel temperature?",

    # --- PHYSICS & CHEMISTRY ---
    "How does sound travel?", "Why does glass break?", "How do shadows change shape?", "Why does metal bend?",
    "How does heat move?", "Why does water boil?", "How do thermometers work?", "Why does ice float?",
    "How does pressure work?", "Why does plastic melt?", "How does static electricity work?", "Why does air take up space?",
    "How does gravity affect water?", "Why does sound echo in caves?", "Why does water stick together?", "How does evaporation cool things?",
    "Why does sound change underwater?", "How does vibration make noise?", "Why do springs bounce back?", "How does friction slow motion?",
    "Why does paper burn?", "How does insulation keep things warm?", "Why do metals conduct electricity?", "How does magnetism travel?",
    "Why do balloons pop?", "How does a prism make a rainbow?", "Why does salt melt ice?", "How does a magnifying glass work?",
    "Why is fire hot?", "How does oil float on water?", "Why do magnets repel each other?", "How does a battery hold energy?",
    "Why do some things sink and others float?", "How does glue work?", "Why is gold heavy?", "How does light travel through fiber optics?",
    "Why does iron rust?", "How do lasers work?", "Why does a flute make music?", "How does radioactivity work?",
    "Why do diamonds sparkle?", "How does smoke rise?", "Why does a compass needle move?", "How do atoms stay together?",
    "Why does cold air sink?", "How does a microwave cook food?", "Why do clothes dry in the sun?", "How do sound-canceling headphones work?",
    "Why is lead used in pencils (but it's actually graphite)?", "How do chemical reactions create heat?", "Why does a curveball curve?",

    # --- ENGINEERING & TECHNOLOGY ---
    "How do bridges stay strong?", "Why do wheels make things move easier?", "How do elevators work?", "Why do engines need fuel?",
    "How do brakes stop cars?", "Why do lights flicker?", "How do traffic lights work?", "Why do clocks tick?",
    "How do doors close softly?", "Why do machines get hot?", "How do windmills make power?", "Why do solar panels work?",
    "How do batteries recharge?", "Why do wires get warm?", "How do remote controls work?", "Why do fans cool us down?",
    "How do touch screens sense fingers?", "Why do alarms make loud sounds?", "How do washing machines clean clothes?",
    "Why do vacuums suck air?", "How do planes fly?", "Why do submarines sink and float?", "How do light bulbs glow?",
    "Why do ships float if they are made of steel?", "How do computers process information?", "Why do sirens change pitch?",
    "How do scanners read barcodes?", "Why do bridges have expansion joints?", "How do zippers work?", "Why do fans have different speeds?",
    "How does a touchscreen work with gloves?", "Why do skyscraper windows not fall out?", "How do rockets steer in space?",
    "Why do car tires have treads?", "How do solar calculators work without batteries?", "Why do speakers vibrate?",
    "How do microphones record sound?", "Why do drones have four propellers?", "How do 3D printers build objects?",
    "Why do electric cars not have tailpipes?", "How do wind turbines turn wind into electricity?", "Why do power lines hiss?",
    "How do wifi signals travel through walls?", "Why do cameras need lenses?", "How do touchless faucets sense hands?",
    "Why do satellites stay in the same spot (geostationary)?", "How do high-speed trains float (Maglev)?",
    "Why do cranes not tip over?", "How do hydraulic jacks lift heavy cars?", "Why do airbags deploy so fast?",

    # --- EVERYDAY LIFE & MATERIALS ---
    "How is rubber made?", "Why does sugar dissolve?", "How is fabric woven?", "Why does bread rise?",
    "How is soap scented?", "Why does metal shine?", "How is paint made?", "Why does paper tear?",
    "How is clay shaped?", "Why does glass reflect light?", "Why do smells spread?", "How do flavors mix?",
    "Why do balloons shrink?", "How does air fill a tire?", "Why do bubbles float upward?", "How does foam form?",
    "Why do candles burn out?", "What is recycling?", "Why do we reuse materials?", "How does trash get sorted?",
    "Why is clean water important?", "How do filters clean water?", "Why do we save energy?", "How do maps show places?",
    "Why do compasses point north?", "How do GPS systems work?", "Why do time zones exist?", "Why does the sky change color at sunset?",
    "How do clouds move?", "Why does frost form?", "How does mist differ from fog?", "Why does snow crunch?",
    "How does hail form?", "Why do puddles disappear?", "How does wind shape rocks?", "Why do leaves fall straight down?",
    "How do waterfalls keep flowing?", "How is cheese aged?", "Why does dough stretch?", "How is juice extracted from fruit?",
    "Why does chocolate melt in hands?", "How is ice cream frozen?", "Why does steam rise upward?", "How is salt collected?",
    "Why does sugar caramelize?", "How is pasta shaped?", "Why does vinegar smell strong?", "How do playground swings move?",
    "Why do seesaws balance?", "How do roller coasters stay on tracks?", "Why do balls roll downhill?", "How do trampolines bounce?",
    "Why do kites stay up?", "How do yo-yos come back?", "Why do spinning tops stay upright?", "How do skateboards turn?",
    "Why do wheels wobble when loose?", "How is glass made from sand?", "Why do sponges absorb water?", "How is paper recycled?",
    "Why does popcorn pop?", "How do matches ignite?", "Why does milk turn sour?", "How is leather made?",
    "Why does honey never spoil?", "How is steel made?", "Why do pens leak in airplanes?", "How is plastic molded?",
    "Why does perfume smell different on different people?", "How is silk produced?", "Why do corks float?", "How are coins minted?",

    # --- PLANTS & ENVIRONMENT ---
    "How do plants breathe?", "Why are leaves green?", "How do seeds know when to grow?", "Why do trees lose leaves in fall?",
    "How do cactus survive in the desert?", "Why do flowers have scents?", "How do vines climb?", "Why do mushrooms grow in the dark?",
    "How do trees get water to their top branches?", "Why do some plants eat insects?", "How does pollination work?",
    "Why do fruits ripen?", "How do roots grow toward water?", "Why do some trees live for thousands of years?",
    "How do plants protect themselves?", "Why do we need forests?", "How do weeds grow so fast?", "Why do sunflowers turn toward the sun?",
    "How do underwater plants get oxygen?", "Why do leaves change color?", "How does bark protect a tree?",
    "Why are some berries poisonous?", "How do plants make seeds?", "Why is compost good for soil?",
    "How do wetlands filter water?", "Why do we have different types of grass?", "How do pinecones open and close?",
    "Why do rainforests have so much rain?", "How do mangrove trees grow in saltwater?", "Why do some flowers only bloom at night?",

    # --- WEATHER & ATMOSPHERE ---
    "What is a tornado?", "How does a blizzard start?", "Why is humid air sticky?", "How do we predict the weather?",
    "Why do we have seasons?", "What is a cold front?", "How does a barometer work?", "Why is the air thinner on mountains?",
    "How does a rainbow form?", "Why do clouds turn gray before rain?", "What is a dust devil?", "How do we track hurricanes?",
    "Why does it smell like rain (petrichor)?", "How do heat waves happen?", "Why is the equator hot?",
    "How does the greenhouse effect work?", "Why do we have wind gusts?", "How does air pressure affect flight?",
    "Why is it colder at the poles?", "How do jet streams move?", "What is the eye of a storm?", "Why do we have seasons?",

    # --- MISC CURIOSITIES ---
    "How do mirrors work?", "Why do we use money?", "How are languages formed?", "Why do people have different accents?",
    "How do books get printed?", "Why do we celebrate birthdays?", "How do movies show motion?", "Why do we wear clothes?",
    "How are roads built?", "Why do we have names?", "How do festivals start?", "Why do we sleep in beds?",
    "How are pencils made?", "Why do we use stamps?", "How do cities get water?", "Why do we have holidays?",
    "How are cartoons animated?", "Why do we have different time zones?", "How is music recorded?", "Why do we use umbrellas?",
    "How do people live in cold climates?", "Why do we have libraries?", "How do we keep food cold?", "Why do we use soap?",
    "How are ships built?", "Why do we have traffic laws?", "How do we measure time?", "Why do we have schools?",
    "How do artists mix colors?", "Why do we have money?", "How do buildings stay cool in summer?", "Why do we use forks and spoons?",
    "How are shoes made?", "Why do we have sports?", "How do we make electricity from dams?", "Why do we use maps?",
    "How do we learn new things?", "Why do we like music?", "How do we stay healthy?", "Why do we have different cultures?",
    "How are bridges designed?", "Why do we have taxes?", "How do we keep the streets clean?", "Why do we use light bulbs?",
    "How are toys designed?", "Why do we have clocks?", "How do we make paper?", "Why do we have different jobs?",
    "How do we tell stories?", "Why do we have rules?", "How do we protect nature?", "Why do we explore?",

    # --- FOOD & KITCHEN SCIENCE ---
    "Why does an onion make you cry?", "How does yeast work?", "Why does salt enhance flavor?", "How do we freeze-dry food?",
    "Why does oil and water not mix?", "How is carbonation added to soda?", "Why does bread get stale?", "How is chocolate made?",
    "Why do apples turn brown?", "How does a pressure cooker work?", "Why does chili feel hot?", "How is butter churned?",
    "Why does cake rise?", "How is vinegar made?", "Why do some foods need to be refrigerated?", "How is decaf coffee made?",
    "Why does gelatin wobble?", "How is maple syrup collected?", "Why does garlic smell so strong?", "How is yogurt made?",

    # --- FINAL ADDITIONS TO REACH 500 ---
    "How do touch-sensitive lamps work?", "Why does a flute sound different than a tuba?", "How do we make glass transparent?",
    "Why do we get hiccups?", "How do birds fly without flapping?", "Why do we have different eye colors?",
    "How do whales communicate across oceans?", "Why does the wind howl?", "How do we filter salt from seawater?",
    "Why do some metals get magnetic?", "How do we make bricks?", "Why do we use concrete?",
    "How do we measure the depth of the ocean?", "Why do we have leap years?", "How do we make ink?",
    "Why does a person’s voice change during puberty?", "How do we make perfume?", "Why do we have different skin tones?",
    "How do we make tires?", "Why do we have eyelashes?", "How do we make medicine?", "Why do we have different hair types?",
    "How do we make electricity from coal?", "Why do we have a heart?", "How do we make glass?", "Why do we have bones?",
    "How do we make clothes?", "Why do we have a brain?", "How do we make plastic?", "Why do we have lungs?",
    "How do we make computers?", "Why do we have a stomach?", "How do we make phones?", "Why do we have a liver?",
    "How do we make cars?", "Why do we have kidneys?", "How do we make planes?", "Why do we have a spleen?",
    "How do we make boats?", "Why do we have a pancreas?", "How do we make trains?", "Why do we have an appendix?",
    "How do we make rockets?", "Why do we have a gallbladder?", "How do we make satellites?", "Why do we have a bladder?",
    "How do we make telescopes?", "Why do we have a throat?", "How do we make microscopes?", "Why do we have a nose?",
    "How do we make cameras?", "Why do we have a mouth?", "How do we make microphones?", "Why do we have an ear?",
    "How do we make speakers?", "Why do we have an eye?", "How do we make televisions?", "Why do we have a head?",
    "How do we make radios?", "Why do we have a neck?", "How do we make clocks?", "Why do we have a shoulder?",
    "How do we make watches?", "Why do we have an arm?", "How do we make jewelry?", "Why do we have a hand?",
    "How do we make furniture?", "Why do we have a finger?", "How do we make tools?", "Why do we have a thumb?",
    "How do we make buildings?", "Why do we have a leg?", "How do we make roads?", "Why do we have a foot?",
    "How do we make bridges?", "Why do we have a toe?", "How do we make tunnels?", "Why do we have a back?",
    "How do we make dams?", "Why do we have a chest?", "How do we make canals?", "Why do we have a waist?",
    "How do we make ports?", "Why do we have a hip?", "How do we make airports?", "Why do we have a knee?",
    "How do we make railways?", "Why do we have an ankle?", "How do we make highways?", "Why do we have a heel?",
    "How do we make subways?", "Why do we have a skin?", "How do we make elevators?", "Why do we have a hair?",
    "How do we make escalators?", "Why do we have a nail?", "How do we make air conditioners?", "Why do we have a tooth?",
    "How do we make heaters?", "Why do we have a tongue?", "How do we make refrigerators?", "Why do we have a lip?",
    "How do we make stoves?", "Why do we have a chin?", "How do we make ovens?", "Why do we have a cheek?",
    "How do we make microwaves?", "Why do we have a forehead?", "How do we make dishwashers?", "Why do we have an eyebrow?",
    "How do we make washing machines?", "Why do we have an eyelid?", "How do we make dryers?", "Why do we have a lash?",
    "How do we make vacuums?", "Why do we have a pore?", "How do we make light bulbs?", "Why do we have a sweat gland?",
    "How do we make batteries?", "Why do we have a nerve?", "How do we make magnets?", "Why do we have a vein?",
    "How do we make wires?", "Why do we have an artery?", "How do we make cables?", "Why do we have a capillary?",
    "How do we make glass?", "Why do we have a cell?", "How do we make ceramics?", "Why do we have a gene?",
    "How do we make polymers?", "Why do we have a DNA?", "How do we make alloys?", "Why do we have a chromosome?"
]

total_samples = 500
random.shuffle(topics)
final_topics = (topics * (total_samples // len(topics) + 1))[:total_samples]

num_styles = len(STYLES)

# UTILS
def save_wav(filename, raw_data, rate=24000):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(rate)
        wf.writeframes(raw_data)

print(f"🚀 Initializing Build: Random Voices & Random Styles")

# MAIN LOOP
for i in range(total_samples):
    topic = final_topics[i]

    selected_style = random.choice(STYLES)

    selected_voice = random.choice(VOICES)
    v_info = VOICE_MAP[selected_voice]

    timestamp = datetime.now().strftime("%H%M%S")
    sample_id = f"teacher_{i:03d}_{selected_style}_{timestamp}"
    audio_path = os.path.join(VOICE_DIR, f"{sample_id}.wav")

    pace = "slow" if selected_style in ["slow", "whispering", "gentle", "calm", "sad", "cry"] else "moderate"
    if selected_style in ["energetic", "excited", "animated"]:
        pace = "brisk"

    description = (
    f"A [GENDER] speaker delivers a {selected_style} and {pace}-paced speech "
    f"with a {v_info['pitch']} pitch. The audio is very clear, with the speaker "
    f"recorded in a close-up setting, providing a high-quality and professional feel."
    )

    print(f"\n🎬 [{i+1}/{total_samples}] {topic}")
    print(f"🎭 Style: {selected_style} (Random) | 🗣️ Voice: {selected_voice} (Random)")

    # Retry Logic for 503 Overload Errors
    max_retries = 3
    retry_count = 0
    success = False

    while retry_count < max_retries and not success:
        try:
            # TEXT GENERATION
            text_resp = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=f"As a {selected_style} teacher, explain '{topic}' in 2 short, simple sentences for a child, based on this description for the character speaking{description}."
            )
            teacher_text = text_resp.text.strip()

            if len(teacher_text.split()) < 12:
                teacher_text += " This explanation is friendly and easy to understand."

            # AUDIO GENERATION

            # Use description in TTS input
            tts_input = f"{description}\n\n{teacher_text}"

            audio_resp = client.models.generate_content(
                model="gemini-2.5-flash-preview-tts",
                contents=tts_input,
                config=types.GenerateContentConfig(
                    response_modalities=["AUDIO"],
                    speech_config=types.SpeechConfig(
                        voice_config=types.VoiceConfig(
                            prebuilt_voice_config=types.PrebuiltVoiceConfig(
                                voice_name=selected_voice
                            )
                        )
                    )
                )
            )


            audio_bytes = audio_resp.candidates[0].content.parts[0].inline_data.data
            save_wav(audio_path, audio_bytes)
            from IPython.display import Audio, display
            display(Audio(audio_path, rate=24000))


            # SAVE METADATA
            with open(METADATA_FILE, "a") as f:
                f.write(json.dumps({
                    "audio_file": f"voices/{sample_id}.wav",
                    "text": teacher_text,
                    "description": description,
                    "voice_name": selected_voice,
                    "style": selected_style,
                    "topic": topic
                }) + "\n")

            print(f"✅ Saved: {sample_id}.wav")
            success = True
            print("⚠️ Waiting 15 seconds")

            time.sleep(15)


        except Exception as e:
            error_str = str(e).lower()
            if "503" in error_str or "overloaded" in error_str:
                retry_count += 1
                wait = retry_count * 15
                print(f"⚠️ Server Overloaded. Waiting {wait}s... (Retry {retry_count}/{max_retries})")
                time.sleep(wait)
            elif "429" in error_str or "quota" in error_str:
                print("🛑 Quota hit → rotating API key")
                rotate_key()
                retry_count += 1
                time.sleep(5)
            else:
                print(f"❌ Error: {e}")
                break